# Make Recommendation

In [1]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
import seaborn as sns
import numpy as np
DEBUG=0

In [2]:
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, Imputer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder
)

In [3]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()

In [4]:
def make_simple_poi(org_type):
    if org_type == 'Plaque':
        return org_type
    elif org_type == 'Monument':
        return 'Art'
    else:
        return 'Building'


In [5]:
def clean_build_year(year):
    if year == None or len(year) < 4:
        return ''
    strip_words = ['unknown', 'circa ', 'abt ', 'about']
    for word in strip_words:
        year=year.replace(word, '')
    return year[0:4]

In [6]:

sql='''SELECT poi.*, styl.style, arct.architect_name, pcat.category
FROM points_of_interest poi
LEFT JOIN architectural_styles styl on (styl.poi_id = poi.poi_id)
LEFT JOIN architects arct on (arct.poi_id = poi.poi_id)
LEFT JOIN poi_categories pcat on (pcat.poi_id = poi.poi_id)
order by poi.poi_id
'''
# pd.DataFrame(db.execute(sql))
# #res.first()
df = pd.read_sql_query(sql, db)
df.head()

,poi_id,name,build_year,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,source,details,style,architect_name,category
0,2,22 Chestnut Park,1905,None,22 Chestnut Park Rosedale Toronto,43.679380,-79.388100,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,South Rosedale Heritage Conservation District,Residential,Detached house,http://www.acotoronto.ca/,"First Occupant: Falconbridge, John D.\r\r\n\r\...",American colonial,Alfred E. Boultbee,None
1,8,43 Cross Street,unknown,None,43 Cross Street Weston York,39.962598,-76.727745,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Weston Heritage Conservation District,Residential,Detached house,http://www.acotoronto.ca/,None,Arts and Crafts,None,None
2,9,Alexander Gemmell House,1889,None,181 Ellis Avenue Swansea Toronto,43.642920,-79.470770,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Heritage property,Residential,Detached house,http://www.acotoronto.ca/,None,Arts and Crafts,John Gemmell,None
3,13,15-17 Gifford Street,unknown,None,15-17 Gifford Street Cabbagetown Toronto,43.663300,-79.363120,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Cabbagetown South Heritage Conservation District,Residential,Semi-detached house,http://www.acotoronto.ca/,None,Arts and Crafts,None,None
4,14,18 Gifford Street,unknown,None,18 Gifford Street Cabbagetown Toronto,43.663150,-79.363600,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Cabbagetown South Heritage Conservation District,Residential,Detached house,http://www.acotoronto.ca/,None,Arts and Crafts,None,None


In [7]:

df['cleaned_year']=df['build_year'].apply(lambda x: clean_build_year(x))


In [8]:

df['cleaned_year']=pd.to_numeric(df['cleaned_year'],errors='coerce',downcast='integer')


In [9]:
df['build_decade']= df['cleaned_year'].apply(lambda x: x//10*10 )


In [10]:
df['poi_type_simple'] = df['poi_type'].apply(lambda x: make_simple_poi(x))

In [11]:
poi_mapper = DataFrameMapper([
   #('Date',None),
    # drop block and address
   # (['poi_id'], None),
    ('build_decade',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     ('category',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     #('architect_name',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     ('style',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     ('poi_type_simple',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
    # ('current_use',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
  #  (['latitude'],None),
  #  (['longitude'],None)
], df_out=True)

In [12]:

poi_mapper.fit(df)
df_features= poi_mapper.transform(df)
df_features.head()
# drop build_decade_0.0
# keywords

,build_decade_1790.0,build_decade_1810.0,build_decade_1820.0,build_decade_1830.0,build_decade_1840.0,build_decade_1850.0,build_decade_1860.0,build_decade_1870.0,build_decade_1880.0,build_decade_1890.0,...,style_Romanesque revival,style_Sculptural,style_Second empire,style_Shingle style,style_Spanish colonial,style_Toronto Bay and Gable,style_Workers Cottage,poi_type_simple_Art,poi_type_simple_Building,poi_type_simple_Plaque
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
df_features.shape

(4686, 105)

In [14]:
user_prefs = np.zeros(105)
# for i in [2,16, 20, 40, 90, 92]:
#     user_prefs[i] = 1
for i in [2,3,4,5,6,7,8,104]:
    user_prefs[i] = 1
user_prefs.shape
type(user_prefs)

numpy.ndarray

In [15]:
df_user=pd.DataFrame(user_prefs).T#, columns=df_features.columns)
df_user.columns = df_features.columns
#df_user=pd.DataFrame.from_records(user_prefs,)
df_user.head()

,build_decade_1790.0,build_decade_1810.0,build_decade_1820.0,build_decade_1830.0,build_decade_1840.0,build_decade_1850.0,build_decade_1860.0,build_decade_1870.0,build_decade_1880.0,build_decade_1890.0,...,style_Romanesque revival,style_Sculptural,style_Second empire,style_Shingle style,style_Spanish colonial,style_Toronto Bay and Gable,style_Workers Cottage,poi_type_simple_Art,poi_type_simple_Building,poi_type_simple_Plaque
0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(df_features,df_user)

In [17]:
cosine_sim[:,0]

array([0.       , 0.       , 0.1767767, ..., 0.1767767, 0.1767767,
       0.1767767])

In [18]:
user_matches = pd.DataFrame(cosine_sim, columns=['user_match'])
user_matches.sort_values('user_match', ascending=False, inplace=True)


In [19]:
#user_matches.iloc[0:20,:]
df['sim_rating'] = 0
for ix,row in user_matches.iloc[0:20,:].iterrows():
    # now find matches close to target of interest
    print(df.iloc[ix,1])
    df.loc[ix,'sim_rating'] = row.user_match
   # print(ix)
df.sort_values('sim_rating', inplace=True, ascending=False)

Women at the University of Toronto
Loring-Wyle Studio
Milne House
Lucius O'Brien (1832-1899)
University of Toronto St. George Campus; University College
University of Toronto St. George Campus; University College
University of Toronto St. George Campus; University College
University of Toronto St. George Campus; University College
University of Toronto St. George Campus; University College
Little Trinity Church
Little Trinity Church
Little Trinity Church
Little Trinity Church
Maple Leaf Gardens
St. James´ Cathedral
St. James´ Cathedral
St. James´ Cathedral
Downsview United Church
Maple Leaf Gardens 1931-2011
Massey Hall


In [ ]:
# profile 1: architecture lover from 18000s
# profile 2: modern architecture
# profile 3: history buff interested in people and sports
# profile 4: art lover

### Distance Method 2: scipy

In [ ]:
from scipy import spatial
d1 = [3,5,5,3,3,2]
d2 = [1,1,3,1,3,2]
#weight_of_importance = [0.05,0.05,0.1,0.1,0.4,0.3] #,0841
#weight_of_importance = [0.1,0.1,0.2,0.2,0.1,0.3] #.1062
weight_of_importance = [0.3,0.3,0.1,0.1,0.1,0.1] #.164 greater distance

result = spatial.distance.cosine(d1, d2, weight_of_importance)
print(result)

In [ ]:
np_features = df_features.as_matrix()
type(np_features)
len(np_features)

In [ ]:
site_prefs=[]
for i in range(0,len(np_features)):
    np_features[i,:]
    result = spatial.distance.cosine(np_features[i,:], user_prefs)#, weight_of_importance)
    res_dict = {'ix': i, 'dist': result}
    site_prefs.append(res_dict)
df_site_prefs = pd.DataFrame(site_prefs)

In [ ]:
# site_prefs.sort()
# site_prefs[0:20]
df_site_prefs.sort_values('dist', inplace=True)
df_site_prefs.head()

In [ ]:
#user_matches.iloc[0:20,:]
for ix,row in df_site_prefs.iloc[0:20,:].iterrows():
    # now find matches close to target of interest
    print(df.iloc[ix,1])
   # print(ix)

### Test from a point

In [20]:
starting_lat =43.657847
starting_long= -79.399801
walk_duration = 1 # in hours
num_points = 15
max_distance = 1000 # meters
num_pois_visit = 20

In [21]:

geopy.distance.geodesic((54.277828,-0.410680), (43.657847,-79.399801)).meters

5589455.712869959

In [22]:
def find_points_in_area(df, lat, long, num_points, max_distance):
    avail_points = []
    
    found_points =0
    
    for ix, row in df.iterrows():

        curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).meters
       # print(curr_pt)
        
        if curr_pt<= max_distance:
            print(row['latitude'], row['longitude'], curr_pt)
            my_dict={}
           # my_dict = df.iloc[ix,:].to_dict()
            my_dict =row.to_dict()
            my_dict['dist_start'] = curr_pt
            avail_points.append(my_dict)
            found_points +=1
            if found_points > num_points:
                break
    df_2 = pd.DataFrame(avail_points)
    return df_2

In [23]:
df_filtered = find_points_in_area(df, starting_lat, starting_long, num_points, max_distance)

43.66307 -79.39517 690.1380220915438
43.66254 -79.3953 635.3671780951389
43.66254 -79.3953 635.3671780951389
43.66254 -79.3953 635.3671780951389
43.66254 -79.3953 635.3671780951389
43.66254 -79.3953 635.3671780951389
43.65056 -79.39552 880.2050207916602
43.65045 -79.39606 875.5052873362704
43.66458 -79.40069 751.5031715935551
43.65229 -79.39473 740.6356433342673
43.65094 -79.39588 830.0375633512373
43.65092 -79.39628 820.3692614967989
43.65105 -79.39568 825.1185713118876
43.66473 -79.40134 774.7499235662036
43.653 -79.39117 880.21688076196
43.653 -79.39117 880.21688076196


In [24]:
df_filtered

,address,architect_name,build_decade,build_year,category,cleaned_year,current_use,demolished_year,details,dist_start,...,image_url,latitude,longitude,name,poi_id,poi_type,poi_type_simple,sim_rating,source,style
0,None,None,NaN,None,Women,NaN,None,None,"This 1985 Toronto Historical Board plaque, loc...",690.138022,...,None,43.66307,-79.39517,Women at the University of Toronto,8549,Plaque,Plaque,0.176777,http://torontoplaques.com,None
1,15 King's College Circle University of Toronto...,Eric R. Arthur,1850.0,1856,None,1856.0,Educational,None,This building was restored after a fire in 189...,635.367178,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66254,-79.39530,University of Toronto St. George Campus; Unive...,1117,Low-rise,Building,0.176777,http://www.acotoronto.ca/,Gothic revival
2,15 King's College Circle University of Toronto...,David B. Dick,1850.0,1856,None,1856.0,Educational,None,This building was restored after a fire in 189...,635.367178,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66254,-79.39530,University of Toronto St. George Campus; Unive...,1117,Low-rise,Building,0.176777,http://www.acotoronto.ca/,Gothic revival
3,15 King's College Circle University of Toronto...,Cumberland and Storm,1850.0,1856,None,1856.0,Educational,None,This building was restored after a fire in 189...,635.367178,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66254,-79.39530,University of Toronto St. George Campus; Unive...,1117,Low-rise,Building,0.176777,http://www.acotoronto.ca/,Gothic revival
4,15 King's College Circle University of Toronto...,"Wilson Newton Roberts Duncan, Architects",1850.0,1856,None,1856.0,Educational,None,This building was restored after a fire in 189...,635.367178,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66254,-79.39530,University of Toronto St. George Campus; Unive...,1117,Low-rise,Building,0.176777,http://www.acotoronto.ca/,Gothic revival
5,15 King's College Circle University of Toronto...,Oxley and Bishop,1850.0,1856,None,1856.0,Educational,None,This building was restored after a fire in 189...,635.367178,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66254,-79.39530,University of Toronto St. George Campus; Unive...,1117,Low-rise,Building,0.176777,http://www.acotoronto.ca/,Gothic revival
6,28-38 Phoebe Street Grange Park Toronto,None,1880.0,1883,None,1883.0,Residential,None,None,880.205021,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.65056,-79.39552,28-38 Phoebe Street,3020,Rowhouse,Building,0.000000,http://www.acotoronto.ca/,Second empire
7,40-54 Phoebe Street Grange Park Toronto,None,1880.0,1883,None,1883.0,Residential,None,None,875.505287,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.65045,-79.39606,40-54 Phoebe Street,3021,Rowhouse,Building,0.000000,http://www.acotoronto.ca/,Second empire
8,370 Huron Street University of Toronto Toronto,None,NaN,unknown,None,NaN,Educational,None,None,751.503172,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.66458,-79.40069,University of Toronto St. George Campus; 370 H...,2985,Detached house,Building,0.000000,http://www.acotoronto.ca/,Second empire
9,27-29 Grange Avenue Grange Park Toronto,None,1880.0,1885,None,1885.0,Residential,None,None,740.635643,...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,43.65229,-79.39473,27-29 Grange Avenue,2976,Semi-detached house,Building,0.000000,http://www.acotoronto.ca/,Second empire


In [25]:
#initialize map with a default lat and long
import folium
map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
folium.CircleMarker(
    [starting_lat, starting_long],
    radius=5,
    color='red',
     fill=True,
       fill_color='#3186cc',
       fill_opacity=0.7).add_to(map_clusters)



# loop through dataframe
for lat, lng, name, address in zip(df_filtered['latitude'], df_filtered['longitude'],  df_filtered['name'],df_filtered['address']):
   label = '{} {}'.format(name, address)
   label = folium.Popup(label, parse_html=True)
   folium.CircleMarker(
       [lat, lng],
       radius=5,
       popup=label,
       color='blue',
       fill=True,
       fill_color='#3186cc',
       fill_opacity=0.7).add_to(map_clusters)

map_clusters

# GENETIC ALGORITHM TO PLOT POINTS
* borrowed and slightly adapted from https://github.com/ZWMiller/PythonProjects/blob/master/genetic_algorithms/evolutionary_algorithm_traveling_salesman.ipynb

In [ ]:
# TODO: cluster points

In [ ]:
walk_stops = {}
for ix,row in df_filtered.iterrows():
    walk_stops[ix] = (row['latitude'], row['longitude'])
walk_stops

In [ ]:

from copy import copy
def create_guess(walk_stops):
    """
    Creates a possible path between all cities, returning to the original.
    Input: List of City IDs
    """
    guess = copy(walk_stops)
    np.random.shuffle(guess)
    guess.append(guess[0])
    return list(guess)

create_guess(list(walk_stops.keys()))

In [ ]:

def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

test_generation = create_generation(list(walk_stops.keys()), population=10)
print(test_generation)

In [ ]:
def travel_time_between_points(point_1, point_2):
    # typical walkign speed is 1.4m/sec
    speed = 1.4
    #find dist between 2 points
    dist = geopy.distance.geodesic(point_1, point_2).meters
    # return guess speed in seconds
    return dist * speed

In [ ]:
walk_stops

In [ ]:
def fitness_score(guess):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(walk_stops[point_id], walk_stops[guess[ix+1]])
    return score

def check_fitness(guesses):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess)))
    return fitness_indicator

print(check_fitness(test_generation))

In [ ]:
def get_breeders_from_generation(guesses, take_best_N=10, take_random_N=5, verbose=False, mutation_rate=0.1):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]]
    best_guess = new_generation[0]
    
    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)
    
    # Second, get some random ones for genetic diversity
    for _ in range(take_random_N):
        ix = np.random.randint(len(guesses))
        new_generation.append(guesses[ix])
        
    # No mutations here since the order really matters.
    # If we wanted to, we could add a "swapping" mutation,
    # but in practice it doesn't seem to be necessary
    
    np.random.shuffle(new_generation)
    return new_generation, best_guess

def make_child(parent1, parent2):
    """ 
    Take some values from parent 1 and hold them in place, then merge in values
    from parent2, filling in from left to right with cities that aren't already in 
    the child. 
    """
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
    child = [-99 for _ in parent1]
    
    for ix in range(0, len(list_of_ids_for_parent1)):
        child[ix] = parent1[ix]
    for ix, gene in enumerate(child):
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    child[-1] = child[0]
    return child

def make_children(old_generation, children_per_couple=1):
    """
    Pairs parents together, and makes children for each pair. 
    If there are an odd number of parent possibilities, one 
    will be left out. 
    
    Pairing happens by pairing the first and last entries. 
    Then the second and second from last, and so on.
    """
    mid_point = len(old_generation)//2
    next_generation = [] 
    
    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(make_child(parent, old_generation[-ix-1]))
    return next_generation

In [ ]:
current_generation = create_generation(list(walk_stops.keys()),population=500)
print_every_n_generations = 5

for i in range(100):
    if not i % print_every_n_generations:
        print("Generation %i: "%i, end='')
        print(len(current_generation))
        is_verbose = True
    else:
        is_verbose = False
    breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                        take_best_N=250, take_random_N=100, 
                                                        verbose=is_verbose)
    current_generation = make_children(breeders, children_per_couple=3)


In [ ]:
def evolve_to_solve(current_generation, max_generations, take_best_N, take_random_N,
                    mutation_rate, children_per_couple, print_every_n_generations, verbose=False):
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i: "%i, end='')
            print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False
        breeders, best_guess = get_breeders_from_generation(current_generation, 
                                                            take_best_N=take_best_N, take_random_N=take_random_N, 
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess))
        current_generation = make_children(breeders, children_per_couple=children_per_couple)
    
    return fitness_tracking, best_guess

current_generation = create_generation(list(walk_stops.keys()),population=500)
fitness_tracking, best_guess = evolve_to_solve(current_generation, 100, 150, 70, 0.5, 3, 5, verbose=True)

In [ ]:
def plot_guess(city_coordinates, guess, guess_in_title=True):
    """
    Takes the coordinates of the cities and the guessed path and
    makes a plot connecting the cities in the guessed order
    Input:
    city_coordinate: dictionary of city id, (x,y)
    guess: list of ids in order
    """
    plot_cities(city_coordinates)
    for ix, current_city in enumerate(guess[:-1]):
        x = [city_coordinates[guess[ix]][0],city_coordinates[guess[ix+1]][0]]
        y = [city_coordinates[guess[ix]][1],city_coordinates[guess[ix+1]][1]]
        plt.plot(x,y,'c--',lw=1)
    plt.scatter(city_coordinates[guess[0]][0],city_coordinates[guess[0]][1], marker='x', c='b')   
    if guess_in_title:
        plt.title("Current Guess: [%s]"%(','.join([str(x) for x in guess])))
    else:
        print("Current Guess: [%s]"%(','.join([str(x) for x in guess])))
    


In [ ]:
def plot_cities(city_coordinates, annotate=True):
    """
    Makes a plot of all cities.
    Input: city_coordinates; dictionary of all cities and their coordinates in (x,y) format
    """
    names = []
    x = []
    y = []
    plt.figure(dpi=250)
    for ix, coord in city_coordinates.items():
        names.append(ix)
        x.append(coord[0])
        y.append(coord[1])
        if annotate:
            plt.annotate(ix, xy=(coord[0], coord[1]), xytext=(20, -20),
                        textcoords='offset points', ha='right', va='bottom',
                        bbox=dict(boxstyle='round,pad=0.5', fc='w', alpha=0.5),
                        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    plt.scatter(x,y,c='r',marker='o')

In [ ]:
#best_guess
path = create_guess(list(walk_stops.keys()))
print(path)
plot_guess(walk_stops, best_guess)


In [ ]:
# add order to df.
print(best_guess)
df_filtered['order'] =0
cnt = 0
for ix in best_guess:
    df_filtered.loc[ix,'order'] = cnt
#     print( df_filtered.loc[ix,:] )
#     print("\n")
    cnt +=1
df_filtered

In [ ]:
def make_fitness_tracking_plot(fitness_tracking):
    """
    Given a list of fitness scores, plot it versus the generation number
    """
    plt.figure(dpi=150)
    plt.plot(range(len(fitness_tracking)), fitness_tracking)
    plt.ylabel("Fitness Score")
    plt.xlabel("Generation")
    plt.title("Fitness Evolution");

make_fitness_tracking_plot(fitness_tracking)

In [ ]:
from folium.features import DivIcon
map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
folium.CircleMarker(
    [starting_lat, starting_long],
    radius=5,
    color='red',
     fill=True,
       fill_color='#3186cc',
       fill_opacity=0.7).add_to(map_clusters)



# loop through dataframe
for lat, lng, name, address,order in zip(df_filtered['latitude'], df_filtered['longitude'],  df_filtered['name'],df_filtered['address'], df_filtered['order']):
    label = '{} {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.map.Marker(
    [lat, lng],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
       # html='<b><div style="color:red,font-size: 10pt">{}</div></b>'.format(order),
        html='<b><div style="color:red">{}</div></b>'.format(order),
        )
    ).add_to(map_clusters)
    


map_clusters